In [1]:
import fiftyone as fo
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers.generation import TFGenerationMixin
import matplotlib.pyplot as plt

/home/saashiv/miniconda3/envs/ERP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-26 01:42:45.286525: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-26 01:42:45.434867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [6]:
# Define classes of interest
classes = [
    "Car", "Motorcycle", "Bus", "Boat", "Bicycle", "Truck", "SUV", "Taxi", "Police Van", "Helicopter",
    "Bridge", "Building", "Road", "Crosswalk", "Sidewalk", "Tunnel", "Highway", "Intersection",
    "Walking", "Shouting", "Swimming", "Jumping", "Falling", "Fighting", "Waving",
    "Ladder", "Hammer", "Screwdriver", "Flashlight", "Backpack", "Camera", "Mobile Phone", "Laptop",
    "Water", "Tree", "Mountain", "Beach", "Lightning", "Rain", "Hail",
    "One-Way Sign", "Speed Limit Sign", "Parking Sign", "Exit Sign", "No Entry Sign", "School Crossing Sign", "Pedestrian Sign",
    "Crowd", "Fire Hydrant", "Gas Station", "Bench", "Fence", "Statue", "Playground", "Electric Pole", "Mailbox", "Manhole", "Public Transport Stop", "Kiosk"
]

# Load training set
train_dataset = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=classes,
    max_samples=1000,  # Adjust as needed
)

# Load validation set
val_dataset = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    classes=classes,
    max_samples=200,  # Adjust as needed
)

# Load test set
test_dataset = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="test",
    label_types=["detections"],
    classes=classes,
    max_samples=200,  # Adjust as needed
)

Ignoring invalid classes ['SUV', 'Police Van', 'Bridge', 'Road', 'Crosswalk', 'Sidewalk', 'Tunnel', 'Highway', 'Intersection', 'Walking', 'Shouting', 'Swimming', 'Jumping', 'Falling', 'Fighting', 'Waving', 'Mobile Phone', 'Water', 'Mountain', 'Beach', 'Lightning', 'Rain', 'Hail', 'One-Way Sign', 'Speed Limit Sign', 'Parking Sign', 'Exit Sign', 'No Entry Sign', 'School Crossing Sign', 'Pedestrian Sign', 'Crowd', 'Fire Hydrant', 'Gas Station', 'Fence', 'Statue', 'Playground', 'Electric Pole', 'Mailbox', 'Manhole', 'Public Transport Stop', 'Kiosk']
You can view the available classes via `fiftyone.utils.openimages.get_classes()`
Found 87 images, downloading the remaining 913
 100% |███████████████████| 913/913 [13.5s elapsed, 0s remaining, 61.9 files/s]      
Dataset info written to '/home/saashiv/fiftyone/open-images-v7/info.json'
Loading existing dataset 'open-images-v7-train-1000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Ignoring

In [13]:
df = pd.read_csv(r'/home/saashiv/Code/EMR/data/text_data/raw/911.csv')
df.head(15)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
5,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,2015-12-10 15:39:04,LANSDALE,CANNON AVE & W 9TH ST,1
6,40.182111,-75.127795,LAUREL AVE & OAKDALE AVE; HORSHAM; Station 35...,19044.0,EMS: NAUSEA/VOMITING,2015-12-10 16:46:48,HORSHAM,LAUREL AVE & OAKDALE AVE,1
7,40.217286,-75.405182,COLLEGEVILLE RD & LYWISKI RD; SKIPPACK; Stati...,19426.0,EMS: RESPIRATORY EMERGENCY,2015-12-10 16:17:05,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,1
8,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: SYNCOPAL EPISODE,2015-12-10 16:51:42,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1
9,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,2015-12-10 17:35:41,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1


In [14]:
# Count the occurrences of each unique value in the 'title' column
class_distribution = df['title'].value_counts()

# Display the counts
print(class_distribution)

title
Traffic: VEHICLE ACCIDENT -    148372
Traffic: DISABLED VEHICLE -     47909
Fire: FIRE ALARM                38336
EMS: FALL VICTIM                34676
EMS: RESPIRATORY EMERGENCY      34248
                                ...  
EMS: DISABLED VEHICLE               1
Fire: PRISONER IN CUSTODY           1
Fire: GENERAL WEAKNESS              1
Fire: SUSPICIOUS                    1
Fire: BARRICADED SUBJECT            1
Name: count, Length: 148, dtype: int64


In [15]:
# Define maximum number of samples per class
amount_per_class = 1000  # You can change this

# Empty DataFrame to collect balanced data
balanced_df = pd.DataFrame()

# Loop through each unique class in the 'title' column
for label in df['title'].unique():
    class_subset = df[df['title'] == label]
    count = len(class_subset)
    
    # If the class has fewer samples than 'amount_per_class', skip it
    if count < amount_per_class:
        continue
    
    # If the class has more samples than 'amount_per_class', truncate it
    if count > amount_per_class:
        class_subset = class_subset.sample(amount_per_class)
    
    balanced_df = pd.concat([balanced_df, class_subset], axis=0)

# Reset index for the new balanced DataFrame
balanced_df.reset_index(drop=True, inplace=True)

# Optionally: Save balanced DataFrame to new CSV
# balanced_df.to_csv('balanced_data.csv', index=False)

print("Balanced DataFrame:", balanced_df['title'].value_counts())
print('Balanced class count:', {len(balanced_df['title'].unique())})

Balanced DataFrame: title
EMS: BACK PAINS/INJURY                  1000
EMS: DIABETIC EMERGENCY                 1000
EMS: ABDOMINAL PAINS                    1000
Fire: FIRE INVESTIGATION                1000
EMS: OVERDOSE                           1000
EMS: UNCONSCIOUS SUBJECT                1000
EMS: CHOKING                            1000
EMS: LACERATIONS                        1000
Fire: TRASH/DUMPSTER FIRE               1000
Fire: UNKNOWN TYPE FIRE                 1000
Fire: BUILDING FIRE                     1000
Fire: ELECTRICAL FIRE OUTSIDE           1000
EMS: FEVER                              1000
EMS: ALLERGIC REACTION                  1000
EMS: FRACTURE                           1000
Fire: WOODS/FIELD FIRE                  1000
Fire: FIRE SPECIAL SERVICE              1000
Fire: VEHICLE FIRE                      1000
Traffic: VEHICLE FIRE -                 1000
EMS: EMS SPECIAL SERVICE                1000
Traffic: HAZARDOUS ROAD CONDITIONS -    1000
EMS: DEHYDRATION             

In [16]:
# Assuming df is your DataFrame and it has a column 'title'
# Drop NaN values
balanced_df.dropna(subset=['title'], inplace=True)

# Lowercase and remove punctuations
balanced_df['PreprocessedTitle'] = balanced_df['title'].str.lower().str.replace('[^\w\s]', '')
labels = balanced_df['PreprocessedTitle'].astype('category').cat.codes.tolist()
# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(balanced_df['PreprocessedTitle'], labels, test_size=0.2, random_state=42)

In [17]:
# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))

# Tokenize your text
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Create a PyTorch dataset
class EmergencyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [19]:
# Initialize datasets
train_dataset = EmergencyDataset(train_encodings, y_train)
val_dataset = EmergencyDataset(val_encodings, y_val)


In [21]:
# Initialize the Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

Step,Training Loss
500,1.928200
1000,0.017000
1500,0.004600
2000,0.002400
2500,0.001500
3000,0.001000
3500,0.000700
4000,0.000600
4500,0.000400
5000,0.000400


TrainOutput(global_step=7800, training_loss=0.12550538304046943, metrics={'train_runtime': 284.3727, 'train_samples_per_second': 438.861, 'train_steps_per_second': 27.429, 'total_flos': 577458999936000.0, 'train_loss': 0.12550538304046943, 'epoch': 3.0})

In [23]:
from transformers import AutoModelForSequenceClassification

#trainer.train(resume_from_checkpoint="./results/checkpoint-500")
# Load the model from a checkpoint
model = AutoModelForSequenceClassification.from_pretrained('./results/checkpoint-7500')  # Replace with your checkpoint directory


In [22]:
trainer.train(resume_from_checkpoint="./results/checkpoint-7500")

Step,Training Loss


KeyboardInterrupt: 

In [28]:
print(balanced_df['title'].unique())

['EMS: BACK PAINS/INJURY' 'EMS: DIABETIC EMERGENCY' 'Fire: GAS-ODOR/LEAK'
 'EMS: CARDIAC EMERGENCY' 'EMS: DIZZINESS' 'EMS: HEAD INJURY'
 'EMS: NAUSEA/VOMITING' 'EMS: RESPIRATORY EMERGENCY'
 'EMS: SYNCOPAL EPISODE' 'Traffic: VEHICLE ACCIDENT -'
 'EMS: VEHICLE ACCIDENT' 'Traffic: DISABLED VEHICLE -'
 'Fire: APPLIANCE FIRE' 'EMS: GENERAL WEAKNESS'
 'Fire: CARBON MONOXIDE DETECTOR' 'EMS: UNKNOWN MEDICAL EMERGENCY'
 'EMS: UNRESPONSIVE SUBJECT' 'Fire: VEHICLE ACCIDENT'
 'EMS: ALTERED MENTAL STATUS' 'Fire: FIRE ALARM' 'EMS: CVA/STROKE'
 'Traffic: ROAD OBSTRUCTION -' 'EMS: SUBJECT IN PAIN' 'EMS: HEMORRHAGING'
 'EMS: FALL VICTIM' 'EMS: ASSAULT VICTIM' 'EMS: SEIZURES'
 'EMS: MEDICAL ALERT ALARM' 'EMS: ABDOMINAL PAINS'
 'Fire: FIRE INVESTIGATION' 'EMS: OVERDOSE' 'EMS: UNCONSCIOUS SUBJECT'
 'EMS: CHOKING' 'EMS: LACERATIONS' 'Fire: TRASH/DUMPSTER FIRE'
 'Fire: UNKNOWN TYPE FIRE' 'Fire: BUILDING FIRE'
 'Fire: ELECTRICAL FIRE OUTSIDE' 'EMS: FEVER' 'EMS: ALLERGIC REACTION'
 'EMS: FRACTURE' 'Fire: WOOD

In [32]:
from transformers import pipeline

labels = balanced_df['PreprocessedTitle'].astype('category')
label_codes = labels.cat.codes.tolist()
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

test_string = 'theres a fire in that forest area '

print(nlp(test_string))
output = nlp(test_string)
# Extract the index (84 in this case)
label_index = int(output[0]['label'].split('_')[1])
original_class_name = labels.cat.categories[label_index]
print(original_class_name)

[{'label': 'LABEL_46', 'score': 0.9980169534683228}]
fire: woods/field fire
